In [ ]:
## TDEF.py

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

In [ ]:
exec(open('/home/sharb24/SBB/thural2/record.py').read())


In [ ]:
######################## READ THE TDEF TABLE ###############################
df2 = spark.table('anp_cabbtdct1_working.SBB_TDEF')
df_tdef=df2.toPandas()

In [ ]:
df_tdef.head(5)

In [ ]:
df_tdef.columns=df_tdef.columns.str.lower()

In [ ]:
###################### CHANGE DATE FORMAT ###########################
from datetime import datetime

########## get numeric month
df_tdef['month_dt'] = pd.to_datetime(df_tdef.fundingdate, format='%m/%d/%Y')
df_tdef['month_dt'] = df_tdef['month_dt'].dt.month

import calendar
df_tdef['month_dt'] = df_tdef['month_dt'].apply(lambda x: calendar.month_name[x]).str.lower()


In [ ]:
df_tdef.info()

In [ ]:
##################### PROCESS TDEF DATA #############################

## convert amountfunded to float
df_tdef = df_tdef.astype({'amountfunded': float})

## create a variable to identify the metric
df_tdef1=df_tdef.assign(metric_name='TDEF Volume')
df_tdef2=df_tdef1[['metric_name','asmbloginid','month_dt','amountfunded']]

##rename 'asmbloginid' to acf2_id
df_tdef2.rename(columns = {"asmbloginid":'acf2_id'}, inplace = True) 

## group by logon_id, metric_name, month
df_tdef3=df_tdef2.groupby(['acf2_id','month_dt','metric_name'],as_index=False).aggregate({
                'amountfunded':'sum'  # count widgets
                })

##transpose data
df_tdef4 = df_tdef3.pivot_table(index=['acf2_id','metric_name'], columns=['month_dt'],
                     values='amountfunded', aggfunc='first', fill_value=0).reset_index()


In [ ]:
#### get months
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

################## CHANGE EVERY FY #############################

complementary = [c for c in net_a if c not in df_tdef4]
df_tdef5= pd.concat([net_a[complementary], df_tdef4], axis=1, join='outer')

##delete the extra filler row
df_tdef5= df_tdef5.drop(labels='a', axis=0)
##convert to float
df_tdef5 = df_tdef5.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

##order the vars
df_tdef5=df_tdef5[['metric_name','acf2_id','november','december','january',
           'february','march','april','may','june','july','august','september','october']]


In [ ]:
################################### PROCESS FOR AMSB #############################################
newvarfunc(df_tdef5)

############## merge to hr/alignment to get am/sm cc
df=spark.sql("select acf2_id,employee_name,am_cost_center,am_cost_center_name,am_cost_center_full_name, \
                     sm_cost_center,sm_cost_center_name from \
                     (select *, row_number() over(partition by acf2_id order by record_date desc) as ran \
                     from anp_cabbtdct1_final.HRM_SBB_ALIGNMENT0)b where ran=1")
dfhr=df.toPandas()

## merge 
df_tdef55=pd.merge(df_tdef5,dfhr,how='left',left_on=['acf2_id'],right_on=['acf2_id'])

## replace 0 with NaN so median only counts non-zeros
df_tdef55.replace(0, np.nan, inplace=True)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_amsb_medianq1=df_tdef55.groupby(['metric_name','sm_cost_center'], as_index=False)['q1'].median()
df_amsb_medianq2=df_tdef55.groupby(['metric_name','sm_cost_center'], as_index=False)['q2'].median()
df_amsb_medianq3=df_tdef55.groupby(['metric_name','sm_cost_center'], as_index=False)['q3'].median()
df_amsb_medianq4=df_tdef55.groupby(['metric_name','sm_cost_center'], as_index=False)['q4'].median()
df_amsb_medianytd=df_tdef55.groupby(['metric_name','sm_cost_center'], as_index=False)['ytd'].median()
##rename median
df_amsb_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_amsb_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_amsb_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_amsb_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_amsb_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
tdef_pt1=pd.merge(pd.merge(df_tdef55,df_amsb_medianq1,
                on=['metric_name','sm_cost_center'],
                how='left'),df_amsb_medianq2,on=['metric_name','sm_cost_center'],how='left')

tdef_pt2=pd.merge(pd.merge(tdef_pt1,df_amsb_medianq3,
                on=['metric_name','sm_cost_center'],
                how='left'),df_amsb_medianq4,on=['metric_name','sm_cost_center'],how='left')

tdef_pt3=pd.merge(tdef_pt2,df_amsb_medianytd,on=['metric_name','sm_cost_center'],how="left")


tdef_pt3['record_date']=rec_date
tdef_pt3['level']='AMSB'
tdef_pt3['scorecard_filter']='SBB'

#tdef_pt3=tdef_pt3.update(tdef_pt3[['november','december','january','february','march','april','may','june','july','august',
#                                   'september','october','q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4',
#                                   'q4_benchmark','ytd','ytd_benchmark']].fillna(0))

tdef_amsb=tdef_pt3[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
############################# CREATE THE AM OUTPUT ##########################################

## aggregate
tdef110=amgrpfunc(tdef_amsb)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(tdef110)

## replace 0 with NaN so median only counts non-zeros
tdef110.replace(0, np.nan, inplace=True)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_am_medianq1=tdef110.groupby(['metric_name'], as_index=False)['q1'].median()
df_am_medianq2=tdef110.groupby(['metric_name'], as_index=False)['q2'].median()
df_am_medianq3=tdef110.groupby(['metric_name'], as_index=False)['q3'].median()
df_am_medianq4=tdef110.groupby(['metric_name'], as_index=False)['q4'].median()
df_am_medianytd=tdef110.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_am_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_am_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_am_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_am_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_am_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
tdef_pt11=pd.merge(pd.merge(tdef110,df_am_medianq1,
                on=['metric_name'],
                how='left'),df_am_medianq2,on=['metric_name'],how='left')

tdef_pt22=pd.merge(pd.merge(tdef_pt11,df_am_medianq3,
                on=['metric_name'],
                how='left'),df_am_medianq4,on=['metric_name'],how='left')

tdef_pt338=pd.merge(tdef_pt22,df_am_medianytd,on=['metric_name'],how="left")

tdef_pt338['acf2_id']=''
tdef_pt338['level']='AM'
tdef_pt338['scorecard_filter']='SBB'
tdef_pt338['employee_name']=tdef_pt338.am_cost_center_full_name

tdef_am_first=tdef_pt338[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## create a copy for Total
tdef_am_tot=tdef_am_first.copy()
tdef_am_tot['scorecard_filter']='Total'

tdef_am_tot1=tdef_am_tot[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
############################# CREATE THE SM OUTPUT ##########################################

tdef111=smgrpfunc(tdef_amsb)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(tdef111)

## replace 0 with NaN so median only counts non-zeros
tdef111.replace(0, np.nan, inplace=True)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_sm_medianq1=tdef111.groupby(['metric_name'], as_index=False)['q1'].median()
df_sm_medianq2=tdef111.groupby(['metric_name'], as_index=False)['q2'].median()
df_sm_medianq3=tdef111.groupby(['metric_name'], as_index=False)['q3'].median()
df_sm_medianq4=tdef111.groupby(['metric_name'], as_index=False)['q4'].median()
df_sm_medianytd=tdef111.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_sm_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_sm_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_sm_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_sm_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_sm_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
tdef_pt111=pd.merge(pd.merge(tdef111,df_sm_medianq1,
                on=['metric_name'],
                how='left'),df_sm_medianq2,on=['metric_name'],how='left')

tdef_pt222=pd.merge(pd.merge(tdef_pt111,df_sm_medianq3,
                on=['metric_name'],
                how='left'),df_sm_medianq4,on=['metric_name'],how='left')

tdef_pt30=pd.merge(tdef_pt222,df_am_medianytd,on=['metric_name'],how="left")

tdef_pt30['acf2_id']=''
tdef_pt30['level']='SM'
tdef_pt30['scorecard_filter']='SBB'
tdef_pt30['employee_name']=tdef_pt30.sm_cost_center_name
tdef_pt30['am_cost_center']=''
tdef_pt30['am_cost_center_name']=''
tdef_pt30['am_cost_center_full_name']=''

tdef_pt350=tdef_pt30[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## create a copy for Total
tdef_sm_tot=tdef_pt350.copy()
tdef_sm_tot['scorecard_filter']='Total'

tdef_sm_tot1=tdef_sm_tot[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
tdef_sm_tot1

In [ ]:
############################## CREATE THE NATIONAL OFFICE OUTPUT ##################################
tdefnat=natgrpfunc(tdef_amsb)

##CREATE THE Q1,Q2,Q3,Q3,YTD VARIABLES
newvarfunc(tdefnat)

## replace 0 with NaN so median only counts non-zeros
tdefnat.replace(0, np.nan, inplace=True)

### calculate qtd and ytd benchmarks
##CREATE A BENCHMARK (based on quarter median)
df_nso_medianq1=tdefnat.groupby(['metric_name'], as_index=False)['q1'].median()
df_nso_medianq2=tdefnat.groupby(['metric_name'], as_index=False)['q2'].median()
df_nso_medianq3=tdefnat.groupby(['metric_name'], as_index=False)['q3'].median()
df_nso_medianq4=tdefnat.groupby(['metric_name'], as_index=False)['q4'].median()
df_nso_medianytd=tdefnat.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
df_nso_medianq1.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
df_nso_medianq2.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
df_nso_medianq3.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
df_nso_medianq4.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
df_nso_medianytd.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)

### merge to dataframe
tdef_pt111=pd.merge(pd.merge(tdefnat,df_nso_medianq1,
                on=['metric_name'],
                how='left'),df_nso_medianq2,on=['metric_name'],how='left')

tdef_pt222=pd.merge(pd.merge(tdef_pt111,df_nso_medianq3,
                on=['metric_name'],
                how='left'),df_nso_medianq4,on=['metric_name'],how='left')

tdef_nso=pd.merge(tdef_pt222,df_nso_medianytd,on=['metric_name'],how="left")

tdef_nso['acf2_id']=''
tdef_nso['level']='National Office'
tdef_nso['scorecard_filter']='Total'
tdef_nso['employee_name']='National Office'
tdef_nso['am_cost_center']=''
tdef_nso['am_cost_center_name']=''
tdef_nso['am_cost_center_full_name']=''
tdef_nso['sm_cost_center']=''
tdef_nso['sm_cost_center_name']=''
tdef_nso['record_date']=rec_date

tdef_nso1=tdef_nso[['metric_name','level','acf2_id','employee_name','november','december','january',
           'february','march','april','may','june','july','august','september','october',
           'q1','q1_benchmark','q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
           'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
############################ CONCATENATE ALL DATAFRAMES ########################################
tdef_all=pd.concat([tdef_amsb,tdef_am_first,tdef_am_tot1,tdef_pt350,tdef_sm_tot1,tdef_nso1],axis=0)


In [ ]:
## fill with zeroes
tdef_allf=tdef_all.replace(np.nan,0.0)

In [ ]:
#tdef_allf

In [ ]:
from pyspark.sql.types import *
#create schema for your dataframe
schema = StructType([StructField("metric_name", StringType(), True)\
                   ,StructField("level", StringType(), True)\
                   ,StructField("acf2_id", StringType(), True)\
                     ,StructField("employee_name", StringType(), True)\
                     ,StructField("november", FloatType(), True)\
                     ,StructField("december", FloatType(), True)\
                     ,StructField("january", FloatType(), True)\
                     ,StructField("february", FloatType(), True)\
                     ,StructField("march", FloatType(), True)\
                     ,StructField("april", FloatType(), True)\
                     ,StructField("may", FloatType(), True)\
                     ,StructField("june", FloatType(), True)\
                     ,StructField("july", FloatType(), True)\
                     ,StructField("august", FloatType(), True)\
                     ,StructField("september", FloatType(), True)\
                     ,StructField("october", FloatType(), True)\
                     ,StructField("q1", FloatType(), True)\
                     ,StructField("q1_benchmark", FloatType(), True)\
                     ,StructField("q2", FloatType(), True)\
                     ,StructField("q2_benchmark", FloatType(), True)\
                     ,StructField("q3", FloatType(), True)\
                     ,StructField("q3_benchmark", FloatType(), True)\
                    ,StructField("q4", FloatType(), True)\
                     ,StructField("q4_benchmark", FloatType(), True)\
                     ,StructField("ytd", FloatType(), True)\
                     ,StructField("ytd_benchmark", FloatType(), True)\
                     ,StructField("am_cost_center", StringType(), True)\
                     ,StructField("am_cost_center_name", StringType(), True)\
                     ,StructField("am_cost_center_full_name", StringType(), True)\
                     ,StructField("sm_cost_center", StringType(), True)\
                      ,StructField("sm_cost_center_name", StringType(), True)\
                      ,StructField("scorecard_filter", StringType(), True)\
                      ,StructField("record_date", StringType(), True)])

#create spark dataframe using schema
tdeffinal = spark.createDataFrame(tdef_allf,schema=schema)



In [ ]:
## Temp change for F24 roll over
#tdeffinal = tdeffinal.toPandas()
#cols = ['november','december','january',
#           'february','march','april','may','june','july','august','september','october',
#           ]
#for col in cols:
#    tdeffinal[col].values[:] = 0
    
#tdeffinal.head(5)
#tdeffinal=spark.createDataFrame(tdeffinal)

In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
tdeffinal.write.mode("overwrite").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_OUTPUT_TDEF") 


In [ ]:
## END OF PROGRAM